In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as wtoken
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

import tensorflow as tf
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import (Activation, Dropout, Dense, Embedding, Flatten, GlobalMaxPooling1D)
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os, string, gc
from tqdm import tqdm
import tldextract as tld


from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%timeit
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
Num GPUs Available:  1


In [3]:
path = '/media/moh/abunayla/Disaster_Tweets/Notebooks/data/'
features = ['keyword', 'location', 'text']

In [4]:
full_df = pd.read_csv(path + 'train.csv')

train = pd.read_csv(path + 'train.csv', usecols = features)

test = pd.read_csv(path + 'test.csv', usecols = features)

samsub = pd.read_csv(path +'sample_submission.csv')

Y =  pd.read_csv(path +'train.csv', usecols = ['target'])

In [5]:
def extract_keywords(text, start_with_char='#'):
    """
    Take in string.
    Extracts words start with certain chars.
    Removes punctiouations
    Returns extracted words that are longer than 3 chars.
    """
    words=[]
    words.append([word.translate(str.maketrans('', '', string.punctuation)) for word in text.split() if word.startswith(start_with_char) and len(word)>3 ])
    
    return words

In [14]:
def fix_url_type_spaces(text):
    """switch %20 from text with empty space"""
    return text.replace('%20', ' ')

In [15]:
def ext_subdomains(column):
    """
    intent: extracts subdomains from urls by using tldextract lib
    accepts: a dataframe column/pandas series of urls.
    returns: a list of extracted subdomains.
    """
    subdomains = []
    for c in column:
        subdomains.append(tld.extract(c).subdomain)
    return subdomains

In [16]:
domains = ext_subdomains('M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ')

In [18]:
tld.extract('M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ').subdomain

'M1'